In [ ]:
# 主要使用transformers库。 
# 框架主要使用pytorch

In [ ]:
# 处理数据 中文字符---》 数字
# 创建数据集。 把处理好的数据变成pytorch的数据集。 
# 生成模型， 有了transformers库， 一般不需要自己创建模型。
# 训练预测过程。

### 处理数据

In [3]:
# 配置代理
import os

os.environ['http_proxy'] = '127.0.0.1:10809'
os.environ['https_proxy'] = '127.0.0.1:10809'

In [1]:
from transformers import AutoTokenizer

In [4]:
tokenizer = AutoTokenizer.from_pretrained('uer/gpt2-chinese-cluecorpussmall')

In [5]:
tokenizer

BertTokenizerFast(name_or_path='uer/gpt2-chinese-cluecorpussmall', vocab_size=21128, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [6]:
# 编码试算
tokenizer.batch_encode_plus([
    '明朝驿使发,一夜絮征袍.素手抽针冷,那堪把剪刀.裁缝寄远道,几日到临洮.',
    '长安一片月,万户捣衣声.秋风吹不尽,总是玉关情.何日平胡虏,良人罢远征.'
])

{'input_ids': [[101, 3209, 3308, 7731, 886, 1355, 117, 671, 1915, 5185, 2519, 6151, 119, 5162, 2797, 2853, 7151, 1107, 117, 6929, 1838, 2828, 1198, 1143, 119, 6161, 5361, 2164, 6823, 6887, 117, 1126, 3189, 1168, 707, 3826, 119, 102], [101, 7270, 2128, 671, 4275, 3299, 117, 674, 2787, 2941, 6132, 1898, 119, 4904, 7599, 1430, 679, 2226, 117, 2600, 3221, 4373, 1068, 2658, 119, 862, 3189, 2398, 5529, 5989, 117, 5679, 782, 5387, 6823, 2519, 119, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [7]:
import torch


class Dataset(torch.utils.data.Dataset):
    def __init__(self):
        with open('chinese_poems.txt', encoding='utf-8') as f:
            lines = f.readlines()
        lines = [i.strip() for i in lines]
        
        self.lines = lines
        
    def __len__(self):
        return len(self.lines)
    
    def __getitem__(self, i):
        return self.lines[i]
    
dataset = Dataset()
len(dataset), dataset[0]

(304752, '欲出未出光辣达,千山万山如火发.须臾走向天上来,逐却残星赶却月.')

In [8]:
def collate_fn(batch):
    # 把中文编码成数字
    data = tokenizer.batch_encode_plus(batch, padding=True, 
                               truncation=True,
                               max_length=512,
                               return_tensors='pt')
    data['labels'] = data['input_ids'].clone()
    return data

In [9]:
# 数据加载器
loader = torch.utils.data.DataLoader(
    dataset=dataset,
    batch_size=4,
    collate_fn=collate_fn,
    shuffle=True,
    drop_last=True
)

In [10]:
for i, data in enumerate(loader):
    break

In [11]:
i

0

In [11]:
data

{'input_ids': tensor([[ 101, 4819, 5285, 5021, 2419, 1874, 2798, 2397,  117, 4635, 4373, 3517,
          704, 7755, 2347, 2170,  119, 3801, 2226, 2496, 2399, 5468, 7744, 2145,
          117, 5825, 5709, 3198, 5688, 4324, 3341, 4692,  119,  102,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,

In [12]:
len(data['input_ids'])

4

In [13]:
for k, v in data.items():
    print(k, v.shape)

input_ids torch.Size([4, 165])
token_type_ids torch.Size([4, 165])
attention_mask torch.Size([4, 165])
labels torch.Size([4, 165])


### 创建模型

In [14]:
from transformers import AutoModelForCausalLM, GPT2Model

In [15]:
# 加载模型
model = AutoModelForCausalLM.from_pretrained('uer/gpt2-chinese-cluecorpussmall')


In [16]:
# 参数量
sum(i.numel() for i in model.parameters())

102068736

In [17]:
with torch.no_grad():
    out = model(**data)

In [18]:
out['loss'], out['logits'].shape

(tensor(9.5183), torch.Size([4, 165, 21128]))

### 训练

In [19]:
from transformers import AdamW
from transformers.optimization import get_scheduler

In [20]:
def train():
    global model
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = model.to(device)
    
    optimizer = AdamW(model.parameters(), lr=5e-5)
    scheduler = get_scheduler(name='linear',
                             num_warmup_steps=0,
                             num_training_steps=len(loader),
                             optimizer=optimizer)
    
    model.train()
    for i, data in enumerate(loader):
        for k in data.keys():
            data[k] = data[k].to(device)
            
        out = model(**data)
        loss = out['loss']
        
        loss.backward()
        # 梯度裁剪
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        optimizer.step()
        scheduler.step()
        
        optimizer.zero_grad()
        model.zero_grad()
        
        if i % 1000 == 0:
            labels = data['labels'][:, 1:]
            out = out['logits'].argmax(dim=2)[:, :-1]
            select = labels != 0
            labels = labels[select]
            out = out[select]
            del select 
            
            # 计算准确率
            accuracy = (labels == out).sum().item() / labels.numel()
            lr = optimizer.state_dict()['param_groups'][0]['lr']
            print(i, loss.item(), lr, accuracy)

In [21]:
train()

D:\.venv\lib\site-packages\transformers\optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


0 8.524815559387207 4.9999343728671184e-05 0.07857142857142857
1000 3.04933500289917 4.9343072399853e-05 0.2315270935960591
2000 3.7905688285827637 4.868680107103481e-05 0.2193877551020408
3000 2.44938588142395 4.8030529742216624e-05 0.20783132530120482


KeyboardInterrupt: 

In [ ]:
# 保存模型
# model = model.to('cpu')
# torch.save(model, 'model.pt')

In [29]:
def generate(text, row, col, model):
    def generate_loop(data):
        with torch.no_grad():
            out = model(**data)
        
        # [5, b, vocab_size]
        out = out['logits']
        # [5, vocab_size]
        out = out[:, -1]
        
        # [5, vocab_size] -> [5, 50]
        topk_value = torch.topk(out, 50).values
        # [5, 50], [5] -> [5, 1]
        topk_value = topk_value[:, -1].unsqueeze(dim=1)
        
        # 赋值
        out = out.masked_fill(out < topk_value, -float('inf'))
        
        # 不允许写特殊字符
        out[:, tokenizer.sep_token_id] = -float('inf')
        out[:, tokenizer.unk_token_id] = -float('inf')
        out[:, tokenizer.pad_token_id] = -float('inf')
        
        for i in '，。':
            out[:, tokenizer.get_vocab()[i]] = -float('inf')
            
        # [5, vocab_size] -> [5, 1]
        out = out.softmax(dim=1)
        out = out.multinomial(num_samples=1)
        
        # 强制添加标点符号
        c = data['input_ids'].shape[1] / (col + 1)
        if c % 1 == 0:
            if c % 2 == 0:
                out[:, 0] = tokenizer.get_vocab()['。']
            else:
                out[:, 0] = tokenizer.get_vocab()['，']
        
        data['input_ids'] = torch.cat([data['input_ids'], out], dim=1)
        data['attention_mask'] = torch.ones_like(data['input_ids'])
        data['token_type_ids'] = torch.zeros_like(data['input_ids'])
        data['labels'] = data['input_ids'].clone()
        
        if data['input_ids'].shape[1] >= row * col + row + 1:
            return data
        return generate_loop(data)
    
    # 重复三遍
    data = tokenizer.batch_encode_plus([text] * 3, return_tensors='pt')
    data['input_ids'] = data['input_ids'][:, :-1]
    data['attention_mask'] = torch.ones_like(data['input_ids'])
    data['token_type_ids'] = torch.zeros_like(data['input_ids'])
    data['labels'] = data['input_ids'].clone()
    
    data = generate_loop(data)
    
    for i in range(3):
        print(i, tokenizer.decode(data['input_ids'][i]))

In [27]:
model = torch.load('save.model')

In [31]:
generate('秋高气爽', row=4, col=7, model=model)

0 [CLS] 秋 高 气 爽 雁 初 飞 ， 云 树 高 峰 落 叶 稀 。 人 尽 夜 归 山 外 宿 ， 鸡 鸣 霜 月 下 寒 衣 。
1 [CLS] 秋 高 气 爽 木 生 秋 ， 何 处 仙 方 未 可 求 。 莫 遣 夜 猿 催 老 去 ， 东 风 吹 老 上 林 丘 。
2 [CLS] 秋 高 气 爽 早 蝉 喧 ， 清 籁 无 声 响 自 喧 。 野 望 岂 容 云 梦 见 ， 江 涵 应 属 月 华 昏 。
